In [1]:
import pandas as pd 

In [2]:
# Define file paths
excel_file_path = 'chennai_cars.xlsx'
csv_file_path = 'chennai_cars.csv'

# Read the Excel file
df = pd.read_excel(excel_file_path, sheet_name='chennai_cars.csv')  # Adjust sheet_name as needed

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Save without row index


In [3]:
# Define file paths
excel_file_path = 'bangalore_cars.xlsx'
csv_file_path = 'bangalore_cars.csv'

# Read the Excel file
df = pd.read_excel(excel_file_path, sheet_name='bangalore_cars.csv')  # Adjust sheet_name as needed

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Save without row index


In [4]:
# Define file paths
excel_file_path = 'delhi_cars.xlsx'
csv_file_path = 'delhi_cars.csv'

# Read the Excel file
df = pd.read_excel(excel_file_path, sheet_name='delhi_cars.csv')  # Adjust sheet_name as needed

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Save without row index


In [5]:
# Define file paths
excel_file_path = 'jaipur_cars.xlsx'
csv_file_path = 'jaipur_cars.csv'

# Read the Excel file
df = pd.read_excel(excel_file_path, sheet_name='jaipur_cars.csv')  # Adjust sheet_name as needed

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Save without row index


In [6]:
# Define file paths
excel_file_path = 'hyderabad_cars.xlsx'
csv_file_path = 'hyderabad_cars.csv'

# Read the Excel file
df = pd.read_excel(excel_file_path, sheet_name='hyderabad_cars.csv')  # Adjust sheet_name as needed

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Save without row index


In [7]:
# Define file paths
excel_file_path = 'kolkata_cars.xlsx'
csv_file_path = 'kolkata_cars.csv'

# Read the Excel file
df = pd.read_excel(excel_file_path, sheet_name='kolkata_cars.csv')  # Adjust sheet_name as needed

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)  # Save without row index


In [8]:

df_bangalore = pd.read_csv('bangalore_cars.csv')
df_chennai = pd.read_csv('chennai_cars.csv')
df_delhi = pd.read_csv('delhi_cars.csv')
df_hyderabad = pd.read_csv('hyderabad_cars.csv')
df_jaipur = pd.read_csv('jaipur_cars.csv')
df_kolkata = pd.read_csv('kolkata_cars.csv')

### Final codes to split all keys and values as a separated columns for all cities

In [9]:
import pandas as pd
import ast
import json

# Load the CSV file into a DataFrame
df = pd.read_csv('bangalore_cars.csv')

# Display the columns to confirm their presence
print("Columns in the DataFrame:", df.columns)

# ---- Helper Functions ----

# Function to convert a string to a dictionary
def convert_to_dict(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return {}

# Function to flatten a dictionary
def flatten_dict(d):
    flattened = {}
    for k, v in d.items():
        if isinstance(v, dict):
            for sub_k, sub_v in flatten_dict(v).items():
                flattened[f"{k}_{sub_k}"] = sub_v
        else:
            flattened[k] = v
    return flattened

# Function to extract and flatten the nested 'top' dictionary
def extract_flatten_dict(nested_dict):
    if 'top' in nested_dict:
        top_data = nested_dict['top']
        df = pd.DataFrame(top_data)
        df = df.drop(columns=['icon'], errors='ignore')
        return df.set_index('key').T
    else:
        return pd.DataFrame()

# Function to flatten the nested 'top' and 'data' sections
def flatten_nested_data(nested_dict):
    # Flatten 'top' section
    top_df = pd.DataFrame(nested_dict.get('top', []))
    
    # Flatten 'data' section
    data_list = nested_dict.get('data', [])
    data_df_list = []
    for category in data_list:
        category_df = pd.DataFrame(category.get('list', []))
        category_df['heading'] = category.get('heading')
        category_df['subHeading'] = category.get('subHeading')
        data_df_list.append(category_df)
    
    data_df = pd.concat(data_df_list, ignore_index=True) if data_df_list else pd.DataFrame()
    
    # Add commonIcon
    common_icon = nested_dict.get('commonIcon', None)
    if not top_df.empty:
        top_df['commonIcon'] = common_icon
    if not data_df.empty:
        data_df['commonIcon'] = common_icon

    return top_df, data_df

# ---- Process 'new_car_detail' Column ----

# Fetch the 'new_car_detail' column only
new_car_detail_column = df['new_car_detail']

# Apply conversion function to the 'new_car_detail' column
new_car_detail_dicts = new_car_detail_column.apply(convert_to_dict)

# Flatten each dictionary and convert to DataFrame
flattened_df_list = [pd.DataFrame([flatten_dict(d)]) for d in new_car_detail_dicts]
flattened_df = pd.concat(flattened_df_list, ignore_index=True)

# ---- Process 'new_car_overview' Column ----

# Convert the 'new_car_overview' column from string to nested dictionaries
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: convert_to_dict(x))

# Extract and flatten the nested dictionaries
df_list = [extract_flatten_dict(d) for d in df['new_car_overview']]
df_combined = pd.concat(df_list, ignore_index=True)

# ---- Process 'new_car_feature' Column ----

# Convert the 'new_car_feature' column from string to nested dictionaries
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: convert_to_dict(x))

# Apply the flattening function to each row and collect results
top_dfs_feature = []
data_dfs_feature = []

for _, row in df.iterrows():
    top_df_feature, data_df_feature = flatten_nested_data(row['new_car_feature'])
    top_dfs_feature.append(top_df_feature)
    data_dfs_feature.append(data_df_feature)

top_combined_feature = pd.concat(top_dfs_feature, ignore_index=True)
data_combined_feature = pd.concat(data_dfs_feature, ignore_index=True)

# ---- Process 'new_car_specs' Column ----

# Initialize a list to collect key-value pairs with an index reference
data_records = []

# Define the processing function for extracting key-value pairs with an index reference
def extract_key_value_pairs(json_data, index):
    try:
        # Convert JSON-like string into a dictionary
        data = json.loads(json_data.replace("'", "\""))

        # Create a dictionary to hold the values for this record
        record = {'index': index}
        
        # Extract 'top' section key-value pairs
        if 'top' in data:
            for item in data['top']:
                record[item['key']] = item['value']

        # Extract 'data' section key-value pairs
        if 'data' in data:
            for section in data['data']:
                for item in section['list']:
                    record[item['key']] = item['value']
        
        # Add the record to the list
        data_records.append(record)
    
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON data at index {index}: {e}")

# Process only the 'new_car_specs' column
if 'new_car_specs' in df.columns:
    for index, row in df.iterrows():
        json_data = row['new_car_specs']
        if pd.notna(json_data):  # Check for NaN values
            extract_key_value_pairs(json_data, index)

# Create a DataFrame from the list of records
records_df = pd.DataFrame(data_records)

# Drop the 'index' column as it's not needed in the final output
records_df = records_df.drop(columns=['index'], errors='ignore')

# ---- Combine All DataFrames Horizontally ----

# Reset index for consistency
flattened_df.reset_index(drop=True, inplace=True)
df_combined.reset_index(drop=True, inplace=True)
top_combined_feature.reset_index(drop=True, inplace=True)
data_combined_feature.reset_index(drop=True, inplace=True)
records_df.reset_index(drop=True, inplace=True)

# Concatenate all DataFrames horizontally
max_len = max(len(flattened_df), len(df_combined), len(top_combined_feature), len(data_combined_feature), len(records_df))

# Ensure all DataFrames have the same number of rows by reindexing
flattened_df = flattened_df.reindex(range(max_len))
df_combined = df_combined.reindex(range(max_len))
top_combined_feature = top_combined_feature.reindex(range(max_len))
data_combined_feature = data_combined_feature.reindex(range(max_len))
records_df = records_df.reindex(range(max_len))

# Concatenate horizontally
final_df = pd.concat([flattened_df, df_combined, top_combined_feature, data_combined_feature, records_df], axis=1)

# Save the final DataFrame to a CSV file
final_df.to_csv('bangalore_cars_output.csv', index=False)

print("\nCombined DataFrames have been saved to 'bangalore_cars_output.csv'.")


Columns in the DataFrame: Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links'],
      dtype='object')

Combined DataFrames have been saved to 'bangalore_cars_output.csv'.


In [10]:
import pandas as pd
import ast
import json

# Load the CSV file into a DataFrame
df = pd.read_csv('chennai_cars.csv')

# Display the columns to confirm their presence
print("Columns in the DataFrame:", df.columns)

# ---- Helper Functions ----

# Function to convert a string to a dictionary
def convert_to_dict(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return {}

# Function to flatten a dictionary
def flatten_dict(d):
    flattened = {}
    for k, v in d.items():
        if isinstance(v, dict):
            for sub_k, sub_v in flatten_dict(v).items():
                flattened[f"{k}_{sub_k}"] = sub_v
        else:
            flattened[k] = v
    return flattened

# Function to extract and flatten the nested 'top' dictionary
def extract_flatten_dict(nested_dict):
    if 'top' in nested_dict:
        top_data = nested_dict['top']
        df = pd.DataFrame(top_data)
        df = df.drop(columns=['icon'], errors='ignore')
        return df.set_index('key').T
    else:
        return pd.DataFrame()

# Function to flatten the nested 'top' and 'data' sections
def flatten_nested_data(nested_dict):
    # Flatten 'top' section
    top_df = pd.DataFrame(nested_dict.get('top', []))
    
    # Flatten 'data' section
    data_list = nested_dict.get('data', [])
    data_df_list = []
    for category in data_list:
        category_df = pd.DataFrame(category.get('list', []))
        category_df['heading'] = category.get('heading')
        category_df['subHeading'] = category.get('subHeading')
        data_df_list.append(category_df)
    
    data_df = pd.concat(data_df_list, ignore_index=True) if data_df_list else pd.DataFrame()
    
    # Add commonIcon
    common_icon = nested_dict.get('commonIcon', None)
    if not top_df.empty:
        top_df['commonIcon'] = common_icon
    if not data_df.empty:
        data_df['commonIcon'] = common_icon

    return top_df, data_df

# ---- Process 'new_car_detail' Column ----

# Fetch the 'new_car_detail' column only
new_car_detail_column = df['new_car_detail']

# Apply conversion function to the 'new_car_detail' column
new_car_detail_dicts = new_car_detail_column.apply(convert_to_dict)

# Flatten each dictionary and convert to DataFrame
flattened_df_list = [pd.DataFrame([flatten_dict(d)]) for d in new_car_detail_dicts]
flattened_df = pd.concat(flattened_df_list, ignore_index=True)

# ---- Process 'new_car_overview' Column ----

# Convert the 'new_car_overview' column from string to nested dictionaries
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: convert_to_dict(x))

# Extract and flatten the nested dictionaries
df_list = [extract_flatten_dict(d) for d in df['new_car_overview']]
df_combined = pd.concat(df_list, ignore_index=True)

# ---- Process 'new_car_feature' Column ----

# Convert the 'new_car_feature' column from string to nested dictionaries
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: convert_to_dict(x))

# Apply the flattening function to each row and collect results
top_dfs_feature = []
data_dfs_feature = []

for _, row in df.iterrows():
    top_df_feature, data_df_feature = flatten_nested_data(row['new_car_feature'])
    top_dfs_feature.append(top_df_feature)
    data_dfs_feature.append(data_df_feature)

top_combined_feature = pd.concat(top_dfs_feature, ignore_index=True)
data_combined_feature = pd.concat(data_dfs_feature, ignore_index=True)

# ---- Process 'new_car_specs' Column ----

# Initialize a list to collect key-value pairs with an index reference
data_records = []

# Define the processing function for extracting key-value pairs with an index reference
def extract_key_value_pairs(json_data, index):
    try:
        # Convert JSON-like string into a dictionary
        data = json.loads(json_data.replace("'", "\""))

        # Create a dictionary to hold the values for this record
        record = {'index': index}
        
        # Extract 'top' section key-value pairs
        if 'top' in data:
            for item in data['top']:
                record[item['key']] = item['value']

        # Extract 'data' section key-value pairs
        if 'data' in data:
            for section in data['data']:
                for item in section['list']:
                    record[item['key']] = item['value']
        
        # Add the record to the list
        data_records.append(record)
    
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON data at index {index}: {e}")

# Process only the 'new_car_specs' column
if 'new_car_specs' in df.columns:
    for index, row in df.iterrows():
        json_data = row['new_car_specs']
        if pd.notna(json_data):  # Check for NaN values
            extract_key_value_pairs(json_data, index)

# Create a DataFrame from the list of records
records_df = pd.DataFrame(data_records)

# Drop the 'index' column as it's not needed in the final output
records_df = records_df.drop(columns=['index'], errors='ignore')

# ---- Combine All DataFrames Horizontally ----

# Reset index for consistency
flattened_df.reset_index(drop=True, inplace=True)
df_combined.reset_index(drop=True, inplace=True)
top_combined_feature.reset_index(drop=True, inplace=True)
data_combined_feature.reset_index(drop=True, inplace=True)
records_df.reset_index(drop=True, inplace=True)

# Concatenate all DataFrames horizontally
max_len = max(len(flattened_df), len(df_combined), len(top_combined_feature), len(data_combined_feature), len(records_df))

# Ensure all DataFrames have the same number of rows by reindexing
flattened_df = flattened_df.reindex(range(max_len))
df_combined = df_combined.reindex(range(max_len))
top_combined_feature = top_combined_feature.reindex(range(max_len))
data_combined_feature = data_combined_feature.reindex(range(max_len))
records_df = records_df.reindex(range(max_len))

# Concatenate horizontally
final_df = pd.concat([flattened_df, df_combined, top_combined_feature, data_combined_feature, records_df], axis=1)

# Save the final DataFrame to a CSV file
final_df.to_csv('chennai_cars_output.csv', index=False)

print("\nCombined DataFrames have been saved to 'chennai_cars_output.csv'.")


Columns in the DataFrame: Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links'],
      dtype='object')

Combined DataFrames have been saved to 'chennai_cars_output.csv'.


In [11]:
import pandas as pd
import ast
import json

# Load the CSV file into a DataFrame
df = pd.read_csv('delhi_cars.csv')

# Display the columns to confirm their presence
print("Columns in the DataFrame:", df.columns)

# ---- Helper Functions ----

# Function to convert a string to a dictionary
def convert_to_dict(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return {}

# Function to flatten a dictionary
def flatten_dict(d):
    flattened = {}
    for k, v in d.items():
        if isinstance(v, dict):
            for sub_k, sub_v in flatten_dict(v).items():
                flattened[f"{k}_{sub_k}"] = sub_v
        else:
            flattened[k] = v
    return flattened

# Function to extract and flatten the nested 'top' dictionary
def extract_flatten_dict(nested_dict):
    if 'top' in nested_dict:
        top_data = nested_dict['top']
        df = pd.DataFrame(top_data)
        df = df.drop(columns=['icon'], errors='ignore')
        return df.set_index('key').T
    else:
        return pd.DataFrame()

# Function to flatten the nested 'top' and 'data' sections
def flatten_nested_data(nested_dict):
    # Flatten 'top' section
    top_df = pd.DataFrame(nested_dict.get('top', []))
    
    # Flatten 'data' section
    data_list = nested_dict.get('data', [])
    data_df_list = []
    for category in data_list:
        category_df = pd.DataFrame(category.get('list', []))
        category_df['heading'] = category.get('heading')
        category_df['subHeading'] = category.get('subHeading')
        data_df_list.append(category_df)
    
    data_df = pd.concat(data_df_list, ignore_index=True) if data_df_list else pd.DataFrame()
    
    # Add commonIcon
    common_icon = nested_dict.get('commonIcon', None)
    if not top_df.empty:
        top_df['commonIcon'] = common_icon
    if not data_df.empty:
        data_df['commonIcon'] = common_icon

    return top_df, data_df

# ---- Process 'new_car_detail' Column ----

# Fetch the 'new_car_detail' column only
new_car_detail_column = df['new_car_detail']

# Apply conversion function to the 'new_car_detail' column
new_car_detail_dicts = new_car_detail_column.apply(convert_to_dict)

# Flatten each dictionary and convert to DataFrame
flattened_df_list = [pd.DataFrame([flatten_dict(d)]) for d in new_car_detail_dicts]
flattened_df = pd.concat(flattened_df_list, ignore_index=True)

# ---- Process 'new_car_overview' Column ----

# Convert the 'new_car_overview' column from string to nested dictionaries
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: convert_to_dict(x))

# Extract and flatten the nested dictionaries
df_list = [extract_flatten_dict(d) for d in df['new_car_overview']]
df_combined = pd.concat(df_list, ignore_index=True)

# ---- Process 'new_car_feature' Column ----

# Convert the 'new_car_feature' column from string to nested dictionaries
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: convert_to_dict(x))

# Apply the flattening function to each row and collect results
top_dfs_feature = []
data_dfs_feature = []

for _, row in df.iterrows():
    top_df_feature, data_df_feature = flatten_nested_data(row['new_car_feature'])
    top_dfs_feature.append(top_df_feature)
    data_dfs_feature.append(data_df_feature)

top_combined_feature = pd.concat(top_dfs_feature, ignore_index=True)
data_combined_feature = pd.concat(data_dfs_feature, ignore_index=True)

# ---- Process 'new_car_specs' Column ----

# Initialize a list to collect key-value pairs with an index reference
data_records = []

# Define the processing function for extracting key-value pairs with an index reference
def extract_key_value_pairs(json_data, index):
    try:
        # Convert JSON-like string into a dictionary
        data = json.loads(json_data.replace("'", "\""))

        # Create a dictionary to hold the values for this record
        record = {'index': index}
        
        # Extract 'top' section key-value pairs
        if 'top' in data:
            for item in data['top']:
                record[item['key']] = item['value']

        # Extract 'data' section key-value pairs
        if 'data' in data:
            for section in data['data']:
                for item in section['list']:
                    record[item['key']] = item['value']
        
        # Add the record to the list
        data_records.append(record)
    
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON data at index {index}: {e}")

# Process only the 'new_car_specs' column
if 'new_car_specs' in df.columns:
    for index, row in df.iterrows():
        json_data = row['new_car_specs']
        if pd.notna(json_data):  # Check for NaN values
            extract_key_value_pairs(json_data, index)

# Create a DataFrame from the list of records
records_df = pd.DataFrame(data_records)

# Drop the 'index' column as it's not needed in the final output
records_df = records_df.drop(columns=['index'], errors='ignore')

# ---- Combine All DataFrames Horizontally ----

# Reset index for consistency
flattened_df.reset_index(drop=True, inplace=True)
df_combined.reset_index(drop=True, inplace=True)
top_combined_feature.reset_index(drop=True, inplace=True)
data_combined_feature.reset_index(drop=True, inplace=True)
records_df.reset_index(drop=True, inplace=True)

# Concatenate all DataFrames horizontally
max_len = max(len(flattened_df), len(df_combined), len(top_combined_feature), len(data_combined_feature), len(records_df))

# Ensure all DataFrames have the same number of rows by reindexing
flattened_df = flattened_df.reindex(range(max_len))
df_combined = df_combined.reindex(range(max_len))
top_combined_feature = top_combined_feature.reindex(range(max_len))
data_combined_feature = data_combined_feature.reindex(range(max_len))
records_df = records_df.reindex(range(max_len))

# Concatenate horizontally
final_df = pd.concat([flattened_df, df_combined, top_combined_feature, data_combined_feature, records_df], axis=1)

# Save the final DataFrame to a CSV file
final_df.to_csv('delhi_cars_output.csv', index=False)

print("\nCombined DataFrames have been saved to 'delhi_cars_output.csv'.")


Columns in the DataFrame: Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links'],
      dtype='object')

Combined DataFrames have been saved to 'delhi_cars_output.csv'.


In [12]:
import pandas as pd
import ast
import json

# Load the CSV file into a DataFrame
df = pd.read_csv('hyderabad_cars.csv')

# Display the columns to confirm their presence
print("Columns in the DataFrame:", df.columns)

# ---- Helper Functions ----

# Function to convert a string to a dictionary
def convert_to_dict(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return {}

# Function to flatten a dictionary
def flatten_dict(d):
    flattened = {}
    for k, v in d.items():
        if isinstance(v, dict):
            for sub_k, sub_v in flatten_dict(v).items():
                flattened[f"{k}_{sub_k}"] = sub_v
        else:
            flattened[k] = v
    return flattened

# Function to extract and flatten the nested 'top' dictionary
def extract_flatten_dict(nested_dict):
    if 'top' in nested_dict:
        top_data = nested_dict['top']
        df = pd.DataFrame(top_data)
        df = df.drop(columns=['icon'], errors='ignore')
        return df.set_index('key').T
    else:
        return pd.DataFrame()

# Function to flatten the nested 'top' and 'data' sections
def flatten_nested_data(nested_dict):
    # Flatten 'top' section
    top_df = pd.DataFrame(nested_dict.get('top', []))
    
    # Flatten 'data' section
    data_list = nested_dict.get('data', [])
    data_df_list = []
    for category in data_list:
        category_df = pd.DataFrame(category.get('list', []))
        category_df['heading'] = category.get('heading')
        category_df['subHeading'] = category.get('subHeading')
        data_df_list.append(category_df)
    
    data_df = pd.concat(data_df_list, ignore_index=True) if data_df_list else pd.DataFrame()
    
    # Add commonIcon
    common_icon = nested_dict.get('commonIcon', None)
    if not top_df.empty:
        top_df['commonIcon'] = common_icon
    if not data_df.empty:
        data_df['commonIcon'] = common_icon

    return top_df, data_df

# ---- Process 'new_car_detail' Column ----

# Fetch the 'new_car_detail' column only
new_car_detail_column = df['new_car_detail']

# Apply conversion function to the 'new_car_detail' column
new_car_detail_dicts = new_car_detail_column.apply(convert_to_dict)

# Flatten each dictionary and convert to DataFrame
flattened_df_list = [pd.DataFrame([flatten_dict(d)]) for d in new_car_detail_dicts]
flattened_df = pd.concat(flattened_df_list, ignore_index=True)

# ---- Process 'new_car_overview' Column ----

# Convert the 'new_car_overview' column from string to nested dictionaries
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: convert_to_dict(x))

# Extract and flatten the nested dictionaries
df_list = [extract_flatten_dict(d) for d in df['new_car_overview']]
df_combined = pd.concat(df_list, ignore_index=True)

# ---- Process 'new_car_feature' Column ----

# Convert the 'new_car_feature' column from string to nested dictionaries
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: convert_to_dict(x))

# Apply the flattening function to each row and collect results
top_dfs_feature = []
data_dfs_feature = []

for _, row in df.iterrows():
    top_df_feature, data_df_feature = flatten_nested_data(row['new_car_feature'])
    top_dfs_feature.append(top_df_feature)
    data_dfs_feature.append(data_df_feature)

top_combined_feature = pd.concat(top_dfs_feature, ignore_index=True)
data_combined_feature = pd.concat(data_dfs_feature, ignore_index=True)

# ---- Process 'new_car_specs' Column ----

# Initialize a list to collect key-value pairs with an index reference
data_records = []

# Define the processing function for extracting key-value pairs with an index reference
def extract_key_value_pairs(json_data, index):
    try:
        # Convert JSON-like string into a dictionary
        data = json.loads(json_data.replace("'", "\""))

        # Create a dictionary to hold the values for this record
        record = {'index': index}
        
        # Extract 'top' section key-value pairs
        if 'top' in data:
            for item in data['top']:
                record[item['key']] = item['value']

        # Extract 'data' section key-value pairs
        if 'data' in data:
            for section in data['data']:
                for item in section['list']:
                    record[item['key']] = item['value']
        
        # Add the record to the list
        data_records.append(record)
    
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON data at index {index}: {e}")

# Process only the 'new_car_specs' column
if 'new_car_specs' in df.columns:
    for index, row in df.iterrows():
        json_data = row['new_car_specs']
        if pd.notna(json_data):  # Check for NaN values
            extract_key_value_pairs(json_data, index)

# Create a DataFrame from the list of records
records_df = pd.DataFrame(data_records)

# Drop the 'index' column as it's not needed in the final output
records_df = records_df.drop(columns=['index'], errors='ignore')

# ---- Combine All DataFrames Horizontally ----

# Reset index for consistency
flattened_df.reset_index(drop=True, inplace=True)
df_combined.reset_index(drop=True, inplace=True)
top_combined_feature.reset_index(drop=True, inplace=True)
data_combined_feature.reset_index(drop=True, inplace=True)
records_df.reset_index(drop=True, inplace=True)

# Concatenate all DataFrames horizontally
max_len = max(len(flattened_df), len(df_combined), len(top_combined_feature), len(data_combined_feature), len(records_df))

# Ensure all DataFrames have the same number of rows by reindexing
flattened_df = flattened_df.reindex(range(max_len))
df_combined = df_combined.reindex(range(max_len))
top_combined_feature = top_combined_feature.reindex(range(max_len))
data_combined_feature = data_combined_feature.reindex(range(max_len))
records_df = records_df.reindex(range(max_len))

# Concatenate horizontally
final_df = pd.concat([flattened_df, df_combined, top_combined_feature, data_combined_feature, records_df], axis=1)

# Save the final DataFrame to a CSV file
final_df.to_csv('hyderabad_cars_output.csv', index=False)

print("\nCombined DataFrames have been saved to 'hyderabad_cars_output.csv'.")


Columns in the DataFrame: Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links'],
      dtype='object')

Combined DataFrames have been saved to 'hyderabad_cars_output.csv'.


In [13]:
import pandas as pd
import ast
import json

# Load the CSV file into a DataFrame
df = pd.read_csv('jaipur_cars.csv')

# Display the columns to confirm their presence
print("Columns in the DataFrame:", df.columns)

# ---- Helper Functions ----

# Function to convert a string to a dictionary
def convert_to_dict(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return {}

# Function to flatten a dictionary
def flatten_dict(d):
    flattened = {}
    for k, v in d.items():
        if isinstance(v, dict):
            for sub_k, sub_v in flatten_dict(v).items():
                flattened[f"{k}_{sub_k}"] = sub_v
        else:
            flattened[k] = v
    return flattened

# Function to extract and flatten the nested 'top' dictionary
def extract_flatten_dict(nested_dict):
    if 'top' in nested_dict:
        top_data = nested_dict['top']
        df = pd.DataFrame(top_data)
        df = df.drop(columns=['icon'], errors='ignore')
        return df.set_index('key').T
    else:
        return pd.DataFrame()

# Function to flatten the nested 'top' and 'data' sections
def flatten_nested_data(nested_dict):
    # Flatten 'top' section
    top_df = pd.DataFrame(nested_dict.get('top', []))
    
    # Flatten 'data' section
    data_list = nested_dict.get('data', [])
    data_df_list = []
    for category in data_list:
        category_df = pd.DataFrame(category.get('list', []))
        category_df['heading'] = category.get('heading')
        category_df['subHeading'] = category.get('subHeading')
        data_df_list.append(category_df)
    
    data_df = pd.concat(data_df_list, ignore_index=True) if data_df_list else pd.DataFrame()
    
    # Add commonIcon
    common_icon = nested_dict.get('commonIcon', None)
    if not top_df.empty:
        top_df['commonIcon'] = common_icon
    if not data_df.empty:
        data_df['commonIcon'] = common_icon

    return top_df, data_df

# ---- Process 'new_car_detail' Column ----

# Fetch the 'new_car_detail' column only
new_car_detail_column = df['new_car_detail']

# Apply conversion function to the 'new_car_detail' column
new_car_detail_dicts = new_car_detail_column.apply(convert_to_dict)

# Flatten each dictionary and convert to DataFrame
flattened_df_list = [pd.DataFrame([flatten_dict(d)]) for d in new_car_detail_dicts]
flattened_df = pd.concat(flattened_df_list, ignore_index=True)

# ---- Process 'new_car_overview' Column ----

# Convert the 'new_car_overview' column from string to nested dictionaries
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: convert_to_dict(x))

# Extract and flatten the nested dictionaries
df_list = [extract_flatten_dict(d) for d in df['new_car_overview']]
df_combined = pd.concat(df_list, ignore_index=True)

# ---- Process 'new_car_feature' Column ----

# Convert the 'new_car_feature' column from string to nested dictionaries
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: convert_to_dict(x))

# Apply the flattening function to each row and collect results
top_dfs_feature = []
data_dfs_feature = []

for _, row in df.iterrows():
    top_df_feature, data_df_feature = flatten_nested_data(row['new_car_feature'])
    top_dfs_feature.append(top_df_feature)
    data_dfs_feature.append(data_df_feature)

top_combined_feature = pd.concat(top_dfs_feature, ignore_index=True)
data_combined_feature = pd.concat(data_dfs_feature, ignore_index=True)

# ---- Process 'new_car_specs' Column ----

# Initialize a list to collect key-value pairs with an index reference
data_records = []

# Define the processing function for extracting key-value pairs with an index reference
def extract_key_value_pairs(json_data, index):
    try:
        # Convert JSON-like string into a dictionary
        data = json.loads(json_data.replace("'", "\""))

        # Create a dictionary to hold the values for this record
        record = {'index': index}
        
        # Extract 'top' section key-value pairs
        if 'top' in data:
            for item in data['top']:
                record[item['key']] = item['value']

        # Extract 'data' section key-value pairs
        if 'data' in data:
            for section in data['data']:
                for item in section['list']:
                    record[item['key']] = item['value']
        
        # Add the record to the list
        data_records.append(record)
    
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON data at index {index}: {e}")

# Process only the 'new_car_specs' column
if 'new_car_specs' in df.columns:
    for index, row in df.iterrows():
        json_data = row['new_car_specs']
        if pd.notna(json_data):  # Check for NaN values
            extract_key_value_pairs(json_data, index)

# Create a DataFrame from the list of records
records_df = pd.DataFrame(data_records)

# Drop the 'index' column as it's not needed in the final output
records_df = records_df.drop(columns=['index'], errors='ignore')

# ---- Combine All DataFrames Horizontally ----

# Reset index for consistency
flattened_df.reset_index(drop=True, inplace=True)
df_combined.reset_index(drop=True, inplace=True)
top_combined_feature.reset_index(drop=True, inplace=True)
data_combined_feature.reset_index(drop=True, inplace=True)
records_df.reset_index(drop=True, inplace=True)

# Concatenate all DataFrames horizontally
max_len = max(len(flattened_df), len(df_combined), len(top_combined_feature), len(data_combined_feature), len(records_df))

# Ensure all DataFrames have the same number of rows by reindexing
flattened_df = flattened_df.reindex(range(max_len))
df_combined = df_combined.reindex(range(max_len))
top_combined_feature = top_combined_feature.reindex(range(max_len))
data_combined_feature = data_combined_feature.reindex(range(max_len))
records_df = records_df.reindex(range(max_len))

# Concatenate horizontally
final_df = pd.concat([flattened_df, df_combined, top_combined_feature, data_combined_feature, records_df], axis=1)

# Save the final DataFrame to a CSV file
final_df.to_csv('jaipur_cars_output.csv', index=False)

print("\nCombined DataFrames have been saved to 'jaipur_cars_output.csv'.")


Columns in the DataFrame: Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links'],
      dtype='object')

Combined DataFrames have been saved to 'jaipur_cars_output.csv'.


In [14]:
import pandas as pd
import ast
import json

# Load the CSV file into a DataFrame
df = pd.read_csv('kolkata_cars.csv')

# Display the columns to confirm their presence
print("Columns in the DataFrame:", df.columns)

# ---- Helper Functions ----

# Function to convert a string to a dictionary
def convert_to_dict(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return {}

# Function to flatten a dictionary
def flatten_dict(d):
    flattened = {}
    for k, v in d.items():
        if isinstance(v, dict):
            for sub_k, sub_v in flatten_dict(v).items():
                flattened[f"{k}_{sub_k}"] = sub_v
        else:
            flattened[k] = v
    return flattened

# Function to extract and flatten the nested 'top' dictionary
def extract_flatten_dict(nested_dict):
    if 'top' in nested_dict:
        top_data = nested_dict['top']
        df = pd.DataFrame(top_data)
        df = df.drop(columns=['icon'], errors='ignore')
        return df.set_index('key').T
    else:
        return pd.DataFrame()

# Function to flatten the nested 'top' and 'data' sections
def flatten_nested_data(nested_dict):
    # Flatten 'top' section
    top_df = pd.DataFrame(nested_dict.get('top', []))
    
    # Flatten 'data' section
    data_list = nested_dict.get('data', [])
    data_df_list = []
    for category in data_list:
        category_df = pd.DataFrame(category.get('list', []))
        category_df['heading'] = category.get('heading')
        category_df['subHeading'] = category.get('subHeading')
        data_df_list.append(category_df)
    
    data_df = pd.concat(data_df_list, ignore_index=True) if data_df_list else pd.DataFrame()
    
    # Add commonIcon
    common_icon = nested_dict.get('commonIcon', None)
    if not top_df.empty:
        top_df['commonIcon'] = common_icon
    if not data_df.empty:
        data_df['commonIcon'] = common_icon

    return top_df, data_df

# ---- Process 'new_car_detail' Column ----

# Fetch the 'new_car_detail' column only
new_car_detail_column = df['new_car_detail']

# Apply conversion function to the 'new_car_detail' column
new_car_detail_dicts = new_car_detail_column.apply(convert_to_dict)

# Flatten each dictionary and convert to DataFrame
flattened_df_list = [pd.DataFrame([flatten_dict(d)]) for d in new_car_detail_dicts]
flattened_df = pd.concat(flattened_df_list, ignore_index=True)

# ---- Process 'new_car_overview' Column ----

# Convert the 'new_car_overview' column from string to nested dictionaries
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: convert_to_dict(x))

# Extract and flatten the nested dictionaries
df_list = [extract_flatten_dict(d) for d in df['new_car_overview']]
df_combined = pd.concat(df_list, ignore_index=True)

# ---- Process 'new_car_feature' Column ----

# Convert the 'new_car_feature' column from string to nested dictionaries
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: convert_to_dict(x))

# Apply the flattening function to each row and collect results
top_dfs_feature = []
data_dfs_feature = []

for _, row in df.iterrows():
    top_df_feature, data_df_feature = flatten_nested_data(row['new_car_feature'])
    top_dfs_feature.append(top_df_feature)
    data_dfs_feature.append(data_df_feature)

top_combined_feature = pd.concat(top_dfs_feature, ignore_index=True)
data_combined_feature = pd.concat(data_dfs_feature, ignore_index=True)

# ---- Process 'new_car_specs' Column ----

# Initialize a list to collect key-value pairs with an index reference
data_records = []

# Define the processing function for extracting key-value pairs with an index reference
def extract_key_value_pairs(json_data, index):
    try:
        # Convert JSON-like string into a dictionary
        data = json.loads(json_data.replace("'", "\""))

        # Create a dictionary to hold the values for this record
        record = {'index': index}
        
        # Extract 'top' section key-value pairs
        if 'top' in data:
            for item in data['top']:
                record[item['key']] = item['value']

        # Extract 'data' section key-value pairs
        if 'data' in data:
            for section in data['data']:
                for item in section['list']:
                    record[item['key']] = item['value']
        
        # Add the record to the list
        data_records.append(record)
    
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error processing JSON data at index {index}: {e}")

# Process only the 'new_car_specs' column
if 'new_car_specs' in df.columns:
    for index, row in df.iterrows():
        json_data = row['new_car_specs']
        if pd.notna(json_data):  # Check for NaN values
            extract_key_value_pairs(json_data, index)

# Create a DataFrame from the list of records
records_df = pd.DataFrame(data_records)

# Drop the 'index' column as it's not needed in the final output
records_df = records_df.drop(columns=['index'], errors='ignore')

# ---- Combine All DataFrames Horizontally ----

# Reset index for consistency
flattened_df.reset_index(drop=True, inplace=True)
df_combined.reset_index(drop=True, inplace=True)
top_combined_feature.reset_index(drop=True, inplace=True)
data_combined_feature.reset_index(drop=True, inplace=True)
records_df.reset_index(drop=True, inplace=True)

# Concatenate all DataFrames horizontally
max_len = max(len(flattened_df), len(df_combined), len(top_combined_feature), len(data_combined_feature), len(records_df))

# Ensure all DataFrames have the same number of rows by reindexing
flattened_df = flattened_df.reindex(range(max_len))
df_combined = df_combined.reindex(range(max_len))
top_combined_feature = top_combined_feature.reindex(range(max_len))
data_combined_feature = data_combined_feature.reindex(range(max_len))
records_df = records_df.reindex(range(max_len))

# Concatenate horizontally
final_df = pd.concat([flattened_df, df_combined, top_combined_feature, data_combined_feature, records_df], axis=1)

# Save the final DataFrame to a CSV file
final_df.to_csv('kolkata_cars_output.csv', index=False)

print("\nCombined DataFrames have been saved to 'kolkata_cars_output.csv'.")


Columns in the DataFrame: Index(['new_car_detail', 'new_car_overview', 'new_car_feature',
       'new_car_specs', 'car_links'],
      dtype='object')

Combined DataFrames have been saved to 'kolkata_cars_output.csv'.
